In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append('../..')
import glb
from dgl.nn import GraphConv

metadata_path = "../../examples/pubmed/metadata.json"
task_path = "../../examples/pubmed/task.json"
g = glb.graph.read_glb_graph(metadata_path=metadata_path)
task = glb.task.read_glb_task(task_path=task_path)

dataset = glb.dataloading.combine_graph_and_task(g, task)
g = dataset[0]

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['NodeFeature'].shape[1], 16, dataset._num_labels)

def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['NodeFeature']
    labels = g.ndata['NodeLabel']
    train_mask = g.ndata['train_set']
    val_mask = g.ndata['val_set']
    test_mask = g.ndata['test_set']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

# train with cpu
model = GCN(g.ndata['NodeFeature'].shape[1], 16, dataset._num_labels)
train(g, model)

# train with gpu
# g = g.to('cuda')
# model = GCN(g.ndata['NodeFeature'].shape[1], 16, dataset._num_labels).to('cuda')
# train(g, model)

/home/huangjin/miniconda3/envs/dgl/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PUBMED dataset.
Node classification on PUBMED dataset. Planetoid split.
In epoch 0, loss: 1.790, val acc: 0.144 (best 0.144), test acc: 0.169 (best 0.169)
In epoch 5, loss: 1.643, val acc: 0.318 (best 0.374), test acc: 0.301 (best 0.343)
In epoch 10, loss: 1.462, val acc: 0.410 (best 0.410), test acc: 0.387 (best 0.387)
In epoch 15, loss: 1.283, val acc: 0.450 (best 0.450), test acc: 0.405 (best 0.405)
In epoch 20, loss: 1.129, val acc: 0.544 (best 0.544), test acc: 0.518 (best 0.518)
In epoch 25, loss: 1.000, val acc: 0.706 (best 0.706), test acc: 0.675 (best 0.675)
In epoch 30, loss: 0.885, val acc: 0.738 (best 0.738), test acc: 0.721 (best 0.721)
In epoch 35, loss: 0.780, val acc: 0.742 (best 0.744), test acc: 0.723 (best 0.722)
In epoch 40, loss: 0.680, val acc: 0.736 (best 0.744), test acc: 0.720 (best 0.722)
In epoch 45, loss: 0.589, val acc: 0.740 (best 0.744), test acc: 0.728 (best 0.722)
In epoch 50, loss: 0.506, val acc: 0.754 (best 0.754), test acc: 0.735 (best 0.735)
In epo